# Trips with no results in Google Directions API query

Check a couple of these to see why?

For LA Metro's 910 (Silver), which runs in ExpressLanes, Google Maps may not understand that as a viable path. Some other waypoints may want to be assembled in a certain order, but I'm explicitly not allowing that in the query.

In [1]:
import geopandas as gpd
import glob
import intake
import os
import pandas as pd

import utils

DATA_PATH = "./gmaps_cache/"
catalog = intake.open_catalog("./*.yml")

/opt/conda/lib/python3.9/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [2]:
df = catalog.gmaps_results.read()
df.head(2)

,calitp_itp_id,shape_id,geometry,route_id,trip_id,service_hours,day_name,trip_departure,destination,origin,departure_in_one_year,waypoints,identifier,duration_in_sec,car_duration_hours,competitive
0,4,shp-10-10,"POLYGON ((-122.15978 37.72170, -122.15716 37.7...",10,13277020,0.45,Thursday,2022-01-06 07:08:00,"[37.670239, -122.08718]","[37.721266, -122.160201]",2023-01-05 07:08:00,"[[37.72521, -122.156367], [37.720677, -122.150...",4__10__13277020__shp-10-10,1261.0,0.35,1
1,4,shp-12-14,"POLYGON ((-122.29889 37.87935, -122.30117 37.8...",12,2883020,0.85,Thursday,2022-01-06 22:58:00,"[37.879358, -122.301843]","[37.79442, -122.271546]",2023-01-05 22:58:00,"[[37.796581, -122.275884], [37.805037, -122.27...",4__12__2883020__shp-12-14,3228.0,0.90,1


In [3]:
# Check if there are any identifiers already cached
# Drop those from our df
FILES = [f for f in glob.glob(f"{DATA_PATH}*.json")]
filenames = []
for f in FILES:
    file = f.strip(f'{DATA_PATH}').strip('.json')
    filenames.append(file)

print(f"# rows: {len(df)}")
print(f"# rows with results: {len(df[df.duration_in_sec.notna()])}")
print(f"# rows with no results: {len(df[df.duration_in_sec.isna()])}")

# rows: 1619
# rows with results: 1603
# rows with no results: 16


## Competitive Routes

In [4]:
df.competitive.value_counts()

1    1476
0     143
Name: competitive, dtype: int64

## Trips with no results from Google

In [ ]:
import dotenv
import googlemaps
import os

dotenv.load_dotenv("_env")

GOOGLE_API_KEY = os.environ["GOOGLE_API_KEY"]

gmaps = googlemaps.Client(key=GOOGLE_API_KEY)

In [ ]:
no_results = catalog.gmaps_results.read()
no_results = no_results[no_results.duration_in_sec.isna()]

In [ ]:
for i, (o, d) in enumerate(zip(origin, destination)):
    try:
        result = gmaps.directions(
            o, d, 
            mode='driving', 
            departure_time=departures[i], 
            waypoints=[f"via:{lat},{lon}" for lat, lon in waypoints[i]]
        )

        utils.save_request_json(
            result, identifiers[i], 
            # Using different paths than utils.DATA_PATH, utils.GCS_FILE_PATH
            DATA_PATH = DATA_PATH,
            GCS_FILE_PATH = GCS_FILE_PATH
        )
    except:
        print(f"No result: {identifiers[i]}")